In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import operator as op
import itertools
import matplotlib.pyplot as plt
import numpy.linalg
import cmath
import multiprocessing
import itertools
import scipy.optimize as optimize
from tabulate import tabulate
from documentCentrality import document_centrality
from networkx import algorithms as ag
%matplotlib inline

In [253]:
def keyword_split(df,key):
    df[key]= df[key].str.split("; ", n = 20, expand = False)  
    return df

In [3]:
def node_key_find(kl,label):
    key=kl[kl['keyword']==label]['id'].iloc[0]
    return key

def node_label_find(kl,key):
    label=kl[kl['id']==key]['keyword'].iloc[0]
    return label    

In [210]:
def nodes_intersection(df,kl,t0,t1,t2):
    prelist=[]
    postlist=[]
    pre_df=df[(df['art_year']>=t0)&(df['art_year']<t1)]
    post_df=df[(df['art_year']>=t1)&(df['art_year']<t2)]
    for index, row in pre_df.iterrows():
        for label in row["keyword"]:
            node_id=node_key_find(kl,label)
            prelist.append(node_id)
    for index, row in post_df.iterrows():
        for label in row["keyword"]:
            node_id=node_key_find(kl,label)
            postlist.append(node_id)
    return set(prelist).intersection(set(postlist))

In [242]:
def build_graph(g,df,kl,nodes,t0,t1):
    g_df=df[(df['art_year']>=t0)&(df['art_year']<t1)]
    #node insert
    for index, row in g_df.iterrows():
        for label in row["keyword"]:
            node_id=node_key_find(kl,label)
            if node_id in nodes:
                if g.has_node(node_id):
                    g.nodes[node_id]['art_id'].add(row["art_id"])
                    g.nodes[node_id]['year'].add(row["art_year"])
                    g.nodes[node_id]['title'].add(row["title"])
                else:
                    art_id={row["art_id"]}
                    year={row["art_year"]}
                    title={row["title"]}
                    g.add_node(node_id,art_id=art_id,year=year,title=title)
    #edge insert
    for index, row in g_df.iterrows():
        edges=list(itertools.combinations(row["keyword"], 2))
        for edge in edges:
            node1=node_key_find(kl,edge[0])
            node2=node_key_find(kl,edge[1])
            if (node1 in nodes) and (node2 in nodes) and (node1!=node2):
                if not g.has_edge(node1,node2):
                    g.add_edge(node1,node2)                
    return g

In [222]:
def node_and_article_feature(df,kl,g,t0,t1):
    f_df=df[(df['art_year']>=t0)&(df['art_year']<t1)]
    article_set=[]
    node_set=[]
    for nd,art in g.node(data='art_id'):
        node_set.append(nd)
        for art_s in art:
            article_set.append(art_s)
    article_set= set(article_set)
    print(len(article_set))
    node_set= set(node_set)
    article_index=list(article_set)
    node_index=list(node_set)
    td=np.zeros((len(g.nodes()), len(article_set)))
    for nd,art in g.node(data='art_id'):
        for art_s in art:
            td[node_index.index(nd)][article_index.index(art_s)]=1
                
    #document centrality feature 
    ca,cn=document_centrality(td,20)
    node_feature=pd.DataFrame({'node_index':node_index,'term_centrality':cn})
    article_feature=pd.DataFrame({'article_index':article_index,'article_centrality_on_node':ca})
    return node_feature,article_feature

In [7]:
def classification_data(df,kl,nodes,g_train,g_test):
    train_labels=[]
    train_rows = list(nx.non_edges(g_train))
    for edge in train_rows:
        if g_test.has_edge(edge[0],edge[1]):
            train_labels.append(1)
        else:
            train_labels.append(0)
    train_data = pd.DataFrame({'row_name':train_rows,'label':train_labels})
    test_data = pd.DataFrame({'row_name':list(set(g_test.edges()))})
    return train_data,test_data

In [8]:
def build_feature_set(df,kl,g,feature_frame,t0,t1):
    #node and article feature
    node_feature,article_feature = node_and_article_feature(df,kl,g,t0,t1)

In [249]:
df = pd.read_csv('dataset/apnea-all,3.csv')
key_list = pd.read_csv('dataset/apnea-distinct_keyword.csv')
article_keywords = pd.read_csv('dataset/apnea-art_id,keyword,key_id,3.csv')
sm_df=df[(df['art_year']<2020)]
sm_df=keyword_split(sm_df)

In [250]:
t0=1991
t1=1992
t2=1993
nodes=nodes_intersection(sm_df,key_list,t0,t1,t2)
g=nx.Graph()
g_train=build_graph(g,sm_df,key_list,nodes,t0,t1)
g=nx.Graph()
g_test=build_graph(g,sm_df,key_list,nodes,t1,t2)

In [254]:
print(len(g_train.edges()),len(g_test.edges()),len(g_train.nodes()),len(g_test.nodes()),len(nodes))


31 20 36 36 36


In [252]:
train_data,test_data=classification_data(sm_df,key_list,nodes,g_train,g_test)
build_feature_set(sm_df,key_list,g_train,train_data,t0,t1)
print(len(train_data),len(train_data[(train_data['label']==1)]))

67
599 11


In [55]:
st=0
end=330
cut=4
tds=train_data
coun2=0
coun=[]
for i,j in tds.iterrows():
    coun1=0
    nb1=set(nx.neighbors(g_train,j[0][0]))
    nb2=set(nx.neighbors(g_train,j[0][1]))
    nb1h2=nb1.copy()
    nb2h2=nb2.copy()
    for n2b1 in nb1h2:
        nb=set(nx.neighbors(g_train,n2b1))
        nb1.update(nb)
    for n2b2 in nb2h2:
        nb=set(nx.neighbors(g_train,n2b2))
        nb2.update(nb)    
    for nnb1 in nb1:
        for nnb2 in nb2:
            if(nnb1==nnb2):
                coun1=coun1+1
    if coun1>0:
        coun2=coun2+1
    coun.append(coun1)  
print(coun2,"/",len(tds))

196350 / 325583


In [ ]:
st=0
end=100000
cut=4
tds=train_data[(train_data['label']==0)]
tds=tds[0:end]
coun2=0
for i,j in tds.iterrows():
    coun1=0
    nb1=set(nx.neighbors(g_train,j[0][0]))
    nb2=set(nx.neighbors(g_train,j[0][1]))
    nb1h2=nb1.copy()
    nb2h2=nb2.copy()
    for n2b1 in nb1h2:
        nb=set(nx.neighbors(g_train,n2b1))
        nb1.update(nb)
    for n2b2 in nb2h2:
        nb=set(nx.neighbors(g_train,n2b2))
        nb2.update(nb)
    for nnb in nb1:
        nnb1=set(nx.neighbors(g_train,nnb))
        for nnnb in nnb1:
            nnnb1=set(nx.neighbors(g_train,nnnb))
            if(j[0][1] in nnnb1):
                coun1=coun1+1
    if coun1>0:
        coun2=coun2+1
print(coun2,"/",len(tds))

In [427]:
st=0
end=330
cut=5
tds=train_data[(train_data['label']==1)]
#tds=tds[0:end]
for jj in range(3,cut):
    count=0
    total=0
    for i,j in tds.iterrows():
        paths = list( nx.all_simple_paths(g_train, source=j[0][0], target=j[0][1],cutoff=jj))
        nb1=set(nx.neighbors(g_train,j[0][0]))
        if len(paths)!=0:
            #print(j[0][0],'+',j[0][1],"----",len(paths))
            count=count+1
            total=total+len(paths)
    print(count,"/",end,"----","cut:",jj,"-----","avg:",total/(jj),"----------")

3.1666666666666665 ---- cut: 3 ----- avg: 8050.666666666667 ----------
3.5757575757575757 ---- cut: 4 ----- avg: 102718.75 ----------


In [266]:
end=4*330
tds=train_data[(train_data['label']==0)]
tds=tds[3*330:end]
for jj in range(3,cut):
    count=0
    total=0
    for i,j in tds.iterrows():
        paths = list( nx.all_simple_paths(g_train, source=j[0][0], target=j[0][1],cutoff=jj))
        if len(paths)!=0:
            #print(j[0][0],'+',j[0][1],"----",len(paths))
            count=count+1
            total=total+len(paths)
    print(count/end,"----","cut:",jj,"-----","avg:",total/(jj),"----------")

1.5909090909090908 ---- cut: 3 ----- avg: 21.666666666666668 ----------


In [438]:
tds=train_data[(train_data['label']==0)]
cm_neigh_len=[]
cm_neigh_list=[]
for i,j in train_data.iterrows():
    coun=0
    cm = list(nx.common_neighbors(g_train, j[0][0],j[0][1]))
    cm_neigh_len.append(len(cm))
    cm_neigh_list.append(cm)

In [443]:
train_data['cm']=cm_neigh_len

In [329]:
st=0
end=10000
cut=4
tds=train_data[(train_data['label']==0)]
#tds=tds[0:end]
coun2=0


In [444]:
def discard_negative(train_data,g_train):
    #pref attach
    pref = nx.preferential_attachment(g_train,list(train_data['row_name']))
    pref = list(pref)
    train_data['pref']=np.array(pref)[:,2]
    cm_neigh_len=[]
    cm_neigh_list=[]
    for i,j in train_data.iterrows():
        coun=0
        cm = list(nx.common_neighbors(g_train, j[0][0],j[0][1]))
        cm_neigh_len.append(len(cm))
        cm_neigh_list.append(cm)
    train_data['cm']=cm_neigh_len

In [22]:
pref = nx.preferential_attachment(g_train,list(train_data['row_name']))
pref = list(pref)
train_data['pref']=np.array(pref)[:,2]

In [23]:
cm_neigh_len=[]
cm_neigh_list=[]
for i,j in train_data.iterrows():
    coun=0
    cm = list(nx.common_neighbors(g_train, j[0][0],j[0][1]))
    cm_neigh_len.append(len(cm))
    cm_neigh_list.append(cm)
train_data['cm']=cm_neigh_len

In [56]:
train_data['cm_hop2']=coun

In [406]:
train_data['soundara']=np.array(soundara)[:,2]

In [92]:
l1= len(train_data[((train_data['pref']>0)&(train_data['cm']>1)&(train_data['cm_hop2']>0))& (train_data['label']==0)])
l2= len(train_data[((train_data['pref']>0)&(train_data['cm']>1)&(train_data['cm_hop2']>0))& (train_data['label']==1)])
print(l1,l2)

6943 296


In [45]:
len(train_data[(train_data['cm_hop2']>0)])

1180

In [74]:
sm_df['art_year'].min()

1980

In [170]:
for i,j in train_data[(train_data['label']==1)][0:20].iterrows():
    print (node_label_find(key_list,j[0][0]),"+",node_label_find(key_list,j[0][1]))
    t1=[]
    t2=[]
    for n1 in g_train[j[0][0]]:
        t1.append(j[0][0])
    for n2 in g_train[j[0][1]]:
        t2.append(g_train[j[0][1]][n2]['title'])
    print(t1)
    print("----------------------------------------------------")

osteotomy + surgery
[10248, 10248]
----------------------------------------------------
osteotomy + distraction
[10248, 10248]
----------------------------------------------------
osteotomy + maxilla
[10248, 10248]
----------------------------------------------------
osteotomy + sleep
[10248, 10248]
----------------------------------------------------
osteotomy + apert syndrome
[10248, 10248]
----------------------------------------------------
osteotomy + obstructive
[10248, 10248]
----------------------------------------------------
malocclusion + overjet
[8202]
----------------------------------------------------
sleep position + children
[14349, 14349, 14349, 14349, 14349]
----------------------------------------------------
sleep position + supine position
[14349, 14349, 14349, 14349, 14349]
----------------------------------------------------
paediatric + morbid obesity
[14348]
----------------------------------------------------
paediatric + anaesthesia
[14348]
-----------------

In [171]:
g_train[10248]

AtlasView({5741: {'art_id': {106857}, 'year': {2001}, 'title': {'Mortised genioplasty in the treatment of obstructive sleep apnea An historical perspective and modification of design'}, 'weight': 1}, 13102: {'art_id': {106857}, 'year': {2001}, 'title': {'Mortised genioplasty in the treatment of obstructive sleep apnea An historical perspective and modification of design'}, 'weight': 1}})

In [124]:
sm_df.groupby('art_year').count()

NameError: name 'sm_df' is not defined

In [169]:
from gensim.models import KeyedVectors
from pyemd import emd
from gensim import corpora
from nltk.corpus import stopwords
from gensim.matutils import softcossim
import importlib
importlib.reload(pyemd)

<module 'pyemd' from 'C:\\Users\\faisal\\Anaconda3\\lib\\site-packages\\pyemd\\__init__.py'>

In [15]:
word_vectors = KeyedVectors.load_word2vec_format('pubmed2018_w2v_200D/pubmed2018_w2v_200D.bin', binary=True)

In [150]:
sentence_obama = 'hi world'.lower().split()
sentence_president = 'hellow world'.lower().split()
sentence_orange = 'congenital central hypoventilation syndrome'.lower().split()

In [151]:
documents = [sentence_obama, sentence_president, sentence_orange]
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(document) for document in documents]

In [152]:
# Convert the sentences into bag-of-words vectors.
sentence_obama = dictionary.doc2bow(sentence_obama)
sentence_president = dictionary.doc2bow(sentence_president)
sentence_orange = dictionary.doc2bow(sentence_orange)
similarity_matrix = word_vectors.similarity_matrix(dictionary)

In [163]:
similarity = softcossim(sentence_obama, sentence_president, similarity_matrix)
print('similarity = %.4f' % similarity)

TypeError: softcossim() missing 1 required positional argument: 'similarity_matrix'

In [171]:
word_vectors.wmdistance('sleepiness', 'insomnia')

ImportError: Please install pyemd Python package to compute WMD.

In [183]:
from platform import python_version
print(python_version())

3.7.0


In [162]:
word_vectors.n_similarity('sleepiness'.lower().split(), 'insomnia'.lower().split())

0.7755101093935269

In [181]:
import temp

In [1]:
import theano

In [2]:
import keras

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow'

In [5]:
import importlib

In [6]:
importlib.reload(keras)

NameError: name 'keras' is not defined

In [7]:
import keras

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow'

In [251]:
import pandas as pd
import numpy as np
# Get some time series data
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/timeseries.csv")

In [252]:
df

,Date,A,B,C,D,E,F,G
0,2008-03-18,24.68,164.93,114.73,26.27,19.21,28.87,63.44
1,2008-03-19,24.18,164.89,114.75,26.22,19.07,27.76,59.98
2,2008-03-20,23.99,164.63,115.04,25.78,19.01,27.04,59.61
3,2008-03-25,24.14,163.92,114.85,27.41,19.61,27.84,59.41
4,2008-03-26,24.44,163.45,114.84,26.86,19.53,28.02,60.09
5,2008-03-27,24.38,163.46,115.40,27.09,19.72,28.25,59.62
6,2008-03-28,24.32,163.22,115.56,27.13,19.63,28.24,58.65
7,2008-03-31,24.19,164.02,115.54,26.74,19.55,28.43,59.20
8,2008-04-01,23.81,163.59,115.72,27.82,20.21,29.17,56.18
9,2008-04-02,24.03,163.32,115.11,28.22,20.42,29.38,56.64


In [253]:
input_cols = ["A","B"] 
df['single_input_vector'] = df[['B','C']].apply(tuple, axis=1).apply(list)

In [254]:
df['single_input_vector'] = df.single_input_vector.apply(lambda x: [list(x)])

In [255]:
df['cumulative_input_vectors'] = df.single_input_vector.cumsum()

In [256]:
output_cols=['B','C']
df['output_vector'] = df[output_cols].apply(tuple, axis=1).apply(list)

In [284]:
from keras.preprocessing.sequence import pad_sequences
max_sequence_length = df.cumulative_input_vectors.apply(len).max()

In [285]:
padded_sequences = pad_sequences(df.cumulative_input_vectors.tolist(), max_sequence_length).tolist()

In [286]:
df['padded_input_vectors'] = pd.Series(padded_sequences).apply(np.asarray)

In [287]:
X_train_init = np.asarray(df.padded_input_vectors)
X_train = np.hstack(X_train_init).reshape(len(df),max_sequence_length,len(input_cols))
y_train = np.hstack(np.asarray(df.output_vector)).reshape(len(df),len(output_cols))

In [288]:
input_length = X_train.shape[1]
input_dim = X_train.shape[2]
output_dim = len(y_train[0])

In [291]:
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense
model = Sequential()
model.add(LSTM(4, input_shape=(input_length,input_dim)))
# The max output value is > 1 so relu is used as final activation.
model.add(Dense(output_dim, activation='relu'))

model.compile(loss='mean_squared_error',
              optimizer='sgd',
              metrics=['accuracy'])

In [343]:
#Set batch_size to 7 to show that it doesn't have to be a factor or multiple of your sample size
history = model.fit(X_train, y_train,validation_split=0.3, shuffle=True,
              batch_size=7, epochs=170,
              verbose = 1)

Train on 4 samples, validate on 2 samples
Epoch 1/170
4/4 [==============================] - 5s 1s/step - loss: 0.5050 - acc: 1.0000 - val_loss: 0.6746 - val_acc: 0.5000
Epoch 2/170
4/4 [==============================] - 0s 3ms/step - loss: 0.5038 - acc: 1.0000 - val_loss: 0.6734 - val_acc: 0.5000
Epoch 3/170
4/4 [==============================] - 0s 2ms/step - loss: 0.5026 - acc: 1.0000 - val_loss: 0.6724 - val_acc: 0.5000
Epoch 4/170
4/4 [==============================] - 0s 1ms/step - loss: 0.5015 - acc: 1.0000 - val_loss: 0.6716 - val_acc: 0.5000
Epoch 5/170
4/4 [==============================] - 0s 1ms/step - loss: 0.5005 - acc: 1.0000 - val_loss: 0.6707 - val_acc: 0.5000
Epoch 6/170
4/4 [==============================] - 0s 8ms/step - loss: 0.4995 - acc: 1.0000 - val_loss: 0.6698 - val_acc: 0.5000
Epoch 7/170
4/4 [==============================] - 0s 4ms/step - loss: 0.4985 - acc: 1.0000 - val_loss: 0.6689 - val_acc: 0.5000
Epoch 8/170
4/4 [==============================] - 0s 87

Epoch 64/170
4/4 [==============================] - 0s 502us/step - loss: 0.4471 - acc: 1.0000 - val_loss: 0.6881 - val_acc: 0.5000
Epoch 65/170
4/4 [==============================] - 0s 752us/step - loss: 0.4463 - acc: 1.0000 - val_loss: 0.6885 - val_acc: 0.5000
Epoch 66/170
4/4 [==============================] - 0s 10ms/step - loss: 0.4454 - acc: 1.0000 - val_loss: 0.6889 - val_acc: 0.5000
Epoch 67/170
4/4 [==============================] - 0s 627us/step - loss: 0.4446 - acc: 1.0000 - val_loss: 0.6892 - val_acc: 0.5000
Epoch 68/170
4/4 [==============================] - 0s 627us/step - loss: 0.4437 - acc: 1.0000 - val_loss: 0.6896 - val_acc: 0.5000
Epoch 69/170
4/4 [==============================] - 0s 1ms/step - loss: 0.4430 - acc: 1.0000 - val_loss: 0.6900 - val_acc: 0.5000
Epoch 70/170
4/4 [==============================] - 0s 753us/step - loss: 0.4422 - acc: 1.0000 - val_loss: 0.6906 - val_acc: 0.5000
Epoch 71/170
4/4 [==============================] - 0s 501us/step - loss: 0.441

4/4 [==============================] - 0s 752us/step - loss: 0.3928 - acc: 1.0000 - val_loss: 0.7040 - val_acc: 0.5000
Epoch 127/170
4/4 [==============================] - 0s 752us/step - loss: 0.3919 - acc: 1.0000 - val_loss: 0.7041 - val_acc: 0.5000
Epoch 128/170
4/4 [==============================] - 0s 753us/step - loss: 0.3909 - acc: 1.0000 - val_loss: 0.7042 - val_acc: 0.5000
Epoch 129/170
4/4 [==============================] - 0s 752us/step - loss: 0.3899 - acc: 1.0000 - val_loss: 0.7042 - val_acc: 0.5000
Epoch 130/170
4/4 [==============================] - 0s 751us/step - loss: 0.3890 - acc: 1.0000 - val_loss: 0.7043 - val_acc: 0.5000
Epoch 131/170
4/4 [==============================] - 0s 1ms/step - loss: 0.3880 - acc: 1.0000 - val_loss: 0.7043 - val_acc: 0.5000
Epoch 132/170
4/4 [==============================] - 0s 10ms/step - loss: 0.3871 - acc: 1.0000 - val_loss: 0.7043 - val_acc: 0.5000
Epoch 133/170
4/4 [==============================] - 0s 626us/step - loss: 0.3861 - ac

In [293]:
model.predict(X_train)

array([[163.71799 , 115.03436 ],
       [163.71896 , 115.03505 ],
       [163.71928 , 115.03526 ],
       [163.71939 , 115.03534 ],
       [163.71942 , 115.03537 ],
       [163.71944 , 115.03538 ],
       [163.71944 , 115.03538 ],
       [163.71945 , 115.035385],
       [163.71945 , 115.035385],
       [163.71945 , 115.035385],
       [163.71945 , 115.035385]], dtype=float32)

In [294]:
y_train

array([[164.93, 114.73],
       [164.89, 114.75],
       [164.63, 115.04],
       [163.92, 114.85],
       [163.45, 114.84],
       [163.46, 115.4 ],
       [163.22, 115.56],
       [164.02, 115.54],
       [163.59, 115.72],
       [163.32, 115.11],
       [163.34, 115.17]])

In [316]:
x=np.array([
	[
		[
			1,3,4,1
		],
		[
			3,2,1,2
		]
	],
	[
		[
			4,1,5,4
		],
		[
			6,7,2,1
		]
	],
	[
		[
			5,8,1,7
		],
		[
			9,1,8,4
		]
	],
    [
		[
			2,3,4,1
		],
		[
			5,2,1,2
		]
	],
	[
		[
			4,1,5,4
		],
		[
			6,7,2,1
		]
	],
	[
		[
			9,8,1,7
		],
		[
			1,1,2,4
		]
	]
])

y=np.array([0,0,0,1,0,1])

In [341]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0, random_state = 0)
input_length = X_train.shape[1]
input_dim = X_train.shape[2]
output_dim = 1

In [338]:
model = Sequential()
model.add(LSTM(4, input_shape=(input_length,input_dim)))
# The max output value is > 1 so relu is used as final activation.
model.add(Dense(output_dim, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [339]:
history = model.fit(X_train, y_train,validation_split=0.3,
              batch_size=7, epochs=70,
              verbose = 1)

Train on 2 samples, validate on 2 samples
Epoch 1/70
2/2 [==============================] - 0s 1ms/step - loss: 0.5555 - acc: 1.0000 - val_loss: 0.7462 - val_acc: 0.5000
Epoch 2/70
2/2 [==============================] - 0s 2ms/step - loss: 0.5539 - acc: 1.0000 - val_loss: 0.7452 - val_acc: 0.5000
Epoch 3/70
2/2 [==============================] - 0s 1ms/step - loss: 0.5523 - acc: 1.0000 - val_loss: 0.7442 - val_acc: 0.5000
Epoch 4/70
2/2 [==============================] - 0s 1ms/step - loss: 0.5507 - acc: 1.0000 - val_loss: 0.7431 - val_acc: 0.5000
Epoch 5/70
2/2 [==============================] - 0s 2ms/step - loss: 0.5491 - acc: 1.0000 - val_loss: 0.7420 - val_acc: 0.5000
Epoch 6/70
2/2 [==============================] - 0s 2ms/step - loss: 0.5476 - acc: 1.0000 - val_loss: 0.7410 - val_acc: 0.5000
Epoch 7/70
2/2 [==============================] - 0s 1ms/step - loss: 0.5461 - acc: 1.0000 - val_loss: 0.7399 - val_acc: 0.5000
Epoch 8/70
2/2 [==============================] - 0s 2ms/step 

Epoch 65/70
2/2 [==============================] - 0s 1ms/step - loss: 0.4692 - acc: 1.0000 - val_loss: 0.6788 - val_acc: 0.5000
Epoch 66/70
2/2 [==============================] - 0s 2ms/step - loss: 0.4680 - acc: 1.0000 - val_loss: 0.6781 - val_acc: 0.5000
Epoch 67/70
2/2 [==============================] - 0s 2ms/step - loss: 0.4667 - acc: 1.0000 - val_loss: 0.6774 - val_acc: 0.5000
Epoch 68/70
2/2 [==============================] - 0s 2ms/step - loss: 0.4655 - acc: 1.0000 - val_loss: 0.6768 - val_acc: 0.5000
Epoch 69/70
2/2 [==============================] - 0s 1ms/step - loss: 0.4642 - acc: 1.0000 - val_loss: 0.6762 - val_acc: 0.5000
Epoch 70/70
2/2 [==============================] - 0s 1ms/step - loss: 0.4630 - acc: 1.0000 - val_loss: 0.6756 - val_acc: 0.5000


In [340]:
model.predict(X_test)

array([[0.5806007 ],
       [0.42457026]], dtype=float32)

In [359]:
import itertools
col_names = ["aa","bb", "cc", "dd"]
all_combinations = itertools.chain(*[itertools.combinations(col_names,i+1) for i,_ in enumerate(col_names)])
ff= list(all_combinations)

In [364]:
for i in ff:
    print(list(i))

['aa']
['bb']
['cc']
['dd']
['aa', 'bb']
['aa', 'cc']
['aa', 'dd']
['bb', 'cc']
['bb', 'dd']
['cc', 'dd']
['aa', 'bb', 'cc']
['aa', 'bb', 'dd']
['aa', 'cc', 'dd']
['bb', 'cc', 'dd']
['aa', 'bb', 'cc', 'dd']


In [365]:
t = train_data[1991].columns.values.tolist()

NameError: name 'train_data' is not defined

In [4]:
data2 = 5 * np.random.randn(100) + 51

In [14]:
old_list = pd.read_csv("dataset\obesity keyword to clean.csv")
new_list = pd.read_csv("dataset\ob09-11toclean.csv")

In [16]:
new_list.head()

,old_keyword,count
0,knee society score,3
1,Protein Kinase A,2
2,Pancreatic Cancer,32
3,microbiology,2
4,Minimally Invasive Surgery,25


In [18]:
merged = pd.merge(old_list, new_list, on='old_keyword', how='left')

In [23]:
merged.head()

,old_keyword,new_keyword,count
0,1,1,10.0
1,10 fructose solution,10 fructose solution,NaN
2,11-beta hydroxysteroid dehydrogenase 1,11-beta hydroxysteroid dehydrogenase 1,NaN
3,11b-hsd type 1,11-beta hydroxysteroid dehydrogenase 1,NaN
4,11-hsd,11-beta hydroxysteroid dehydrogenase 1,NaN


In [34]:
merged.loc[merged['count'].isnull()]

,old_keyword,new_keyword,count
1,10 fructose solution,10 fructose solution,NaN
2,11-beta hydroxysteroid dehydrogenase 1,11-beta hydroxysteroid dehydrogenase 1,NaN
3,11b-hsd type 1,11-beta hydroxysteroid dehydrogenase 1,NaN
4,11-hsd,11-beta hydroxysteroid dehydrogenase 1,NaN
5,11hsd1,11-hydroxysteroid dehydrogenase type 1,NaN
6,11-hsd1,11-hydroxysteroid dehydrogenase type 1,NaN
7,11-hsd1 inhibition,11-hydroxysteroid dehydrogenase inhibitor,NaN
8,11-hsd1 inhibitor,11-hydroxysteroid dehydrogenase inhibitor,NaN
9,11-hydroxysteroid,11-hydroxysteroid,NaN
10,11-hydroxysteroid dehydrogenase,11-hydroxysteroid dehydrogenase,NaN
